In [1]:
import pyntcloud
import pickle
import tensorflow as tf
import numpy as np
import importlib
from train import provider
from train import box_util
import tf_util #for conv2d
from model_util import NUM_HEADING_BIN, NUM_SIZE_CLUSTER, NUM_OBJECT_POINT
from model_util import point_cloud_masking, get_center_regression_net
from model_util import placeholder_inputs, parse_output_to_tensors, get_loss
from model_util import huber_loss
import datetime
import logging
from random import shuffle
import pandas as pd
from tensorflow.python import debug as tf_debug
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
from tensorflow.python import debug
from pyntcloud import PyntCloud

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Mon, 11 June 2018 04:25:02


In [2]:
importlib.reload(box_util)
importlib.reload(provider)

<module 'train.provider' from '/datasets/home/90/490/scgullap/ece_285_proj/frustum-pointnets/train/provider.py'>

In [3]:
# NUM_POINT=2048
# TRAIN_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='train',\
#                             rotate_to_center=True, random_flip=False, random_shift=False, one_hot=True)

In [4]:
# VAL_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='val',\
#                             rotate_to_center=True, random_flip=False, random_shift=False, one_hot=True)

In [5]:
# def indices(window_len, stride_len, point_set, labels, corners_gt):
#     IOU = []
#     start = []
#     end = []
#     IOU_1 = []
#     IOU_2 = []
#     corner = []
#     j = np.min(point_set,0)[2]
#     i = j+window_len
#     idtuple=set()
#     while i<=np.max(point_set,0)[2]:
#             max_ind = np.searchsorted(point_set[:,2], i)
#             min_ind = np.searchsorted(point_set[:,2], j)
#             if (min_ind,max_ind) not in idtuple and max_ind-min_ind>5:
#                 #print('fgbd')
#                 idtuple.add((min_ind,max_ind))
#                 labels_window = np.zeros_like(point_set[:,0])
#                 labels_window[min_ind:max_ind] = 1.0
#                 union = np.sum(np.logical_or(labels, labels_window))
#                 inter = np.sum(np.logical_and(labels, labels_window))
#                 if (inter*1.0/union) < 0.3 or (inter*1.0/union) >0.7 :
#                     end.append(max_ind)
#                     start.append(min_ind)
#                     IOU.append(inter*1.0/union)
#             j = j+stride_len
#             i = j+window_len
#     IOU = np.array(IOU)>=0.7
#     return np.array(start),np.array(end), np.array(IOU)

In [6]:
# def get_list(start_idx,end_idx,num_points=1024,ensure_all=True):
#     """
#     param:
#     start_idx  : list of indexes of the starting point in each window
#     end_idx    : list of indees of ending point in each window
#     NOTES: we used two lists because we ran into issues with tensorflow when we used tuples,
#            this is a quick hack to avoid that
#     num_points : number of points to be sampled from i.e number of points in each window
#     ensure_all : in the case that there are fewwer than required points ensures all points are picked atleast once if selected as True
    
    
#     returns:
#     out:  an array of size 
          
#           num_windowsxnum_pointsx1
          
#           this will be used inside tensorflow to divide into windows 
#           after we pass the points trough pointnet
#     """
    
#     out= []
#     for i in range(np.shape(start_idx)[0]):
        
#         if end_idx[i]-start_idx[i]+1<=num_points:
#             indexes=np.arange(start_idx[i],end_idx[i])
#             indexes2=np.random.choice(indexes, num_points-np.shape(indexes)[0],replace=True)
#             indexes=np.concatenate((indexes,indexes2))
#         else:
#             indexes=np.random.choice(np.arange(start_idx[i],end_idx[i]),num_points,replace=False)
#         out.append(np.reshape(indexes,(-1,1)))
    
#     return np.array(out)


In [7]:
# # https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py
# from tensorflow.python.ops import array_ops

# def focal_loss(sigmoid_p, target_tensor, weights=None, alpha=0.99, gamma=2):
#     r"""Compute focal loss for predictions.
#         Multi-labels Focal loss formula:
#             FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log
#             (1-p)
#                  ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
#     Args:
#      prediction_tensor: A float tensor of shape [batch_size, num_anchors,
#         num_classes] representing the predicted logits for each class
#      target_tensor: A float tensor of shape [batch_size, num_anchors,
#         num_classes] representing one-hot encoded classification targets
#      weights: A float tensor of shape [batch_size, num_anchors]
#      alpha: A scalar tensor for focal loss alpha hyper-parameter
#      gamma: A scalar tensor for focal loss gamma hyper-parameter
#     Returns:
#         loss: A (scalar) tensor representing the value of the loss function
#     """
#     #sigmoid_p = tf.nn.sigmoid(prediction_tensor)
#     zeros = array_ops.zeros_like(target_tensor, dtype=target_tensor.dtype)
    
#     # For poitive prediction, only need consider front part loss, back part is 0;
#     # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
#     pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
#     # For negative prediction, only need consider back part loss, front part is 0;
#     # target_tensor > zeros <=> z=1, so negative coefficient = 0.
#     neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
#     per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
#                           - (1.0 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
#     return tf.reduce_sum(per_entry_cross_ent)

In [8]:
# # def classification_net(point_cloud, label_out_5, label_out_1, label_out_2, label_out_3, label_out_4, idx5, idx1, idx2, idx3, idx4, 
# #                             is_training=True, bn_decay=True):
# def classification_net(point_cloud, label_out_5,  idx5, 
#                             is_training=True, bn_decay=True):
#     '''
#     3D PointNet network.
#     param:
    
#     point_cloud: TF tensor in shape (1,N,4)
#                  frustum point clouds with XYZ and intensity in point channels
#                  XYZs are in frustum coordinate
#     label_out: TF tensor in shape (1,1)
#             length-1 vector indicating the classification as car or background
    
#     ??????????????????????????????????????????????????????????????????????????????????????
    
#     is_training: TF boolean scalar
#     bn_decay: TF float scalar
#     end_points: dict                 SHOULD CHANGE THIS
#     Output:                                         
#     logits: TF tensor in shape (B,N,2), scores for bkg/clutter and object
#     end_points: dict
#     ??????????????????????????????????????????????????????????????????????????????????????
#     '''
    
#     batch_size = point_cloud.get_shape()[0].value #1 in our case
#     num_point = point_cloud.get_shape()[1].value  #N in our case
#     print('shape of point_cloud= ',point_cloud.get_shape())
#     net = tf.expand_dims(point_cloud, 2)
#     print('shape of net= ',net.get_shape())

#     net = tf_util.conv2d(net, 64, [1,1],
#                          padding='VALID', stride=[1,1],
#                          bn=True, is_training=is_training,
#                          scope='conv1', bn_decay=bn_decay)
#     net = tf_util.conv2d(net, 64, [1,1],
#                          padding='VALID', stride=[1,1],
#                          bn=True, is_training=is_training,
#                          scope='conv2', bn_decay=bn_decay)
#     point_feat = tf_util.conv2d(net, 64, [1,1],
#                          padding='VALID', stride=[1,1],
#                          bn=True, is_training=is_training,
#                          scope='conv3', bn_decay=bn_decay)
#     net = tf_util.conv2d(point_feat, 128, [1,1],
#                          padding='VALID', stride=[1,1],
#                          bn=True, is_training=is_training,
#                          scope='conv4', bn_decay=bn_decay)
#     features = tf_util.conv2d(net, 1024, [1,1],
#                          padding='VALID', stride=[1,1],
#                          bn=True, is_training=is_training,
#                          scope='conv5', bn_decay=bn_decay)
#     print('shape of features= ',features.get_shape())
#     features=tf.squeeze(features,axis=2)
#     features=tf.squeeze(features,axis=0)   
#     print('shape of features= ',features.get_shape())
#     #features=tf.placeholder(tf.float32,[1,1024,4])

#     def fn(x):
#         return tf.gather_nd(features,x)
    
#     windows_5 =tf.map_fn(fn,idx5,dtype=tf.float32)
#     print('shape of win5= ',windows_5.get_shape())
#     windows_5_1024 = tf.reduce_max(windows_5,axis=1)
#     print('shape of win51024= ',windows_5_1024.get_shape())
    
    
#     winex_5 =  tf.reshape(windows_5_1024,shape=[-1,1,1,1024])

   
#     winex = tf.concat([winex_5], axis=0)
#     label_out = tf.concat([label_out_5],axis=0)

    
    

#     #classification head
#     d1 = tf_util.conv2d(winex, 512, [1,1],
#                              padding='VALID', stride=[1,1],
#                              bn=True, is_training=is_training,
#                              scope='dense1', bn_decay=bn_decay)
#     d2 = tf_util.conv2d(d1, 256, [1,1],
#                              padding='VALID', stride=[1,1],
#                              bn=True, is_training=is_training,
#                              scope='dense2', bn_decay=bn_decay)
#     d2=tf.layers.dropout(d2,rate=0.5, training=is_training, name='drop1')
    
#     logits = tf_util.conv2d(d2, 1, [1,1],
#                              padding='VALID', stride=[1,1],
#                              bn=True, is_training=is_training,
#                              scope='out', bn_decay=bn_decay, activation_fn=None)
    
#     logits = tf.squeeze(logits)
#     logits=tf.reshape(tf.squeeze(logits),(1,-1))
#     sig = tf.sigmoid(logits)

#     #sig = tf.reshape(tf.squeeze(sig),(-1,1))
#     y=tf.reshape(sig,[-1])
    
#     max_ind = tf.argmax(y)
#     print('shape of index= ',max_ind.get_shape())
    
    
    
#     #window selection for next stage
#     subset=idx_5[max_ind]
#     print('shape of subset= ',subset.get_shape())
#     def fn_select(x):
#         return tf.gather_nd(point_cloud[0],x)
#     selected_win=fn_select(subset)
#     print('shape of selected win= ',selected_win.get_shape())
    

    
    
    
    
    
#     tp = tf.equal(label_out[max_ind], 1)
    
    
#     differentiable_round = tf.maximum(y-0.499,0)
#     differentiable_round = differentiable_round * 10000
#     y_pred = tf.minimum(differentiable_round, 1)
#     #y_pred=tf.round(y)
    
#     true_pos  = tf.reduce_sum(tf.cast(tf.logical_and(tf.cast(label_out,tf.bool),tf.cast(y_pred,tf.bool)),tf.float32))
    
#     false_pos = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(tf.cast(label_out,tf.bool)),tf.cast(y_pred,tf.bool)),tf.float32))
#     true_neg  = tf.reduce_sum(tf.cast(tf.logical_not(tf.logical_or(tf.cast(label_out, tf.bool),\
#                                                                    tf.cast(y_pred, tf.bool))),tf.float32))
#     false_neg = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not\
#                                                      (tf.cast(y_pred,tf.bool)),tf.cast(label_out,tf.bool)),tf.float32))
#     return sig,logits, label_out,true_pos,false_pos,true_neg,false_neg, tp, max_ind,selected_win

In [9]:


def get_center_regression_net(object_point_cloud, one_hot_vec=None,
                              is_training=True, bn_decay=True):
    ''' Regression network for center delta. a.k.a. T-Net.
    Input:
        object_point_cloud: TF tensor in shape (B,M,C)
            point clouds in 3D mask coordinate
        one_hot_vec: TF tensor in shape (B,3)
            length-3 vectors indicating predicted object type
    Output:
        predicted_center: TF tensor in shape (B,3)
    ''' 
    #object_point_cloud=tf.reshape(object_point_cloud,(None,512,3))
    num_point = object_point_cloud.get_shape()[1].value
    net = tf.expand_dims(object_point_cloud, 2)
    #(B,512,1,4)
    
    
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg1-stage1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg2-stage1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 256, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg3-stage1', bn_decay=bn_decay)
    net = tf_util.max_pool2d(net, [num_point,1],
        padding='VALID', scope='maxpool-stage1')
    net = tf.squeeze(net, axis=[1,2])
    #net = tf.concat([net, one_hot_vec], axis=1)
    net = tf_util.fully_connected(net, 256, scope='fc1-stage1', bn=True,
        is_training=is_training, bn_decay=bn_decay)
    net = tf_util.fully_connected(net, 128, scope='fc2-stage1', bn=True,
        is_training=is_training, bn_decay=bn_decay)
    predicted_center = tf_util.fully_connected(net, 3, activation_fn=None,
        scope='fc3-stage1')
    return predicted_center

In [10]:
NUM_HEADING_BIN=6
NUM_SIZE_CLUSTER=1
def get_3d_box_estimation_v1_net(object_point_cloud, one_hot_vec=None,
                                 is_training=True, bn_decay=True):
    ''' 3D Box Estimation PointNet v1 network.
    Input:
        object_point_cloud: TF tensor in shape (B,M,C)
            point clouds in object coordinate
        one_hot_vec: TF tensor in shape (B,3)
            length-3 vectors indicating predicted object type
    Output:
        output: TF tensor in shape (B,3+NUM_HEADING_BIN*2+NUM_SIZE_CLUSTER*4)
            including box centers, heading bin class scores and residuals,
            and size cluster scores and residuals
    ''' 
    #object_point_cloud=tf.reshape(object_point_cloud,(None,512,3))
    num_point = object_point_cloud.get_shape()[1].value
    net = tf.expand_dims(object_point_cloud, 2)
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg2', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 256, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg3', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 512, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv-reg4', bn_decay=bn_decay)
    net = tf_util.max_pool2d(net, [num_point,1],
        padding='VALID', scope='maxpool2')
    net = tf.squeeze(net, axis=[1,2])
    #fonet = tf.concat([net, one_hot_vec], axis=1)
    net = tf_util.fully_connected(net, 512, scope='fc1', bn=True,
        is_training=is_training, bn_decay=bn_decay)
    net = tf_util.fully_connected(net, 256, scope='fc2', bn=True,
        is_training=is_training, bn_decay=bn_decay)

    # The first 3 numbers: box center coordinates (cx,cy,cz),
    # the next NUM_HEADING_BIN*2:  heading bin class scores and bin residuals
    # next sizelbh_residuals: l,b,h residuals
    output = tf_util.fully_connected(net,
        3+3+NUM_HEADING_BIN*2, activation_fn=None, scope='fc3')
    return output

In [11]:

#g_mean_size_arr = np.zeros((NUM_SIZE_CLUSTER, 3))
g_mean_size_arr =  np.array([[0.84422524,0.66068622,1.76255119]])
print(g_mean_size_arr.shape)
def get_box3d_corners_helper(centers, headings, sizes):
    """ TF layer. Input: (N,3), (N,), (N,3), Output: (N,8,3) """
    #print '-----', centers
    N = 5
    l = tf.slice(sizes, [0,0], [-1,1]) # (N,1)
    w = tf.slice(sizes, [0,1], [-1,1]) # (N,1)
    h = tf.slice(sizes, [0,2], [-1,1]) # (N,1)
    #print l,w,h
    x_corners = tf.concat([l/2,l/2,-l/2,-l/2,l/2,l/2,-l/2,-l/2], axis=1) # (N,8)
    y_corners = tf.concat([h/2,h/2,h/2,h/2,-h/2,-h/2,-h/2,-h/2], axis=1) # (N,8)
    z_corners = tf.concat([w/2,-w/2,-w/2,w/2,w/2,-w/2,-w/2,w/2], axis=1) # (N,8)
    corners = tf.concat([tf.expand_dims(x_corners,1), tf.expand_dims(y_corners,1), tf.expand_dims(z_corners,1)], axis=1) # (N,3,8)
    #print x_corners, y_corners, z_corners
    c = tf.cos(headings)
    s = tf.sin(headings)
    ones = tf.ones([N], dtype=tf.float32)
    zeros = tf.zeros([N], dtype=tf.float32)
    row1 = tf.stack([c,zeros,s], axis=1) # (N,3)
    row2 = tf.stack([zeros,ones,zeros], axis=1)
    row3 = tf.stack([-s,zeros,c], axis=1)
    R = tf.concat([tf.expand_dims(row1,1), tf.expand_dims(row2,1), tf.expand_dims(row3,1)], axis=1) # (N,3,3)
    #print row1, row2, row3, R, N
    corners_3d = tf.matmul(R, corners) # (N,3,8)
    corners_3d += tf.tile(tf.expand_dims(centers,2), [1,1,8]) # (N,3,8)
    corners_3d = tf.transpose(corners_3d, perm=[0,2,1]) # (N,8,3)
    return corners_3d

def get_corners2(outputs,heading_class_label,stage1_center, flip=False,test=False):
    centers=outputs[:,:3]+tf.squeeze(stage1_center,axis=1)
    sizes=outputs[:,3:6]+tf.constant(g_mean_size_arr, tf.float32)
    
    heading_bin_centers = tf.constant(np.arange(0,np.pi,np.pi/NUM_HEADING_BIN), dtype=tf.float32) 
    
    def if_true():
        max_ind = tf.argmax(outputs[:,6:12],axis=-1)
        print('shape of max_ind= ',max_ind.get_shape())
        test_onehot = tf.one_hot(max_ind, 6, 1.0, 0.0, axis=1, dtype=tf.float32)
        print('shape of one_hot= ',test_onehot.get_shape())
        #headingclass_angle = max_ind*tf.constant(2*np.pi/NUM_HEADING_BIN, dtype=float32)
        
        if not flip:
            headings=tf.reduce_sum(tf.multiply(tf.to_float(test_onehot),outputs[:,12:]+tf.reshape(heading_bin_centers,(1,6))),axis=1)
        else:
            headings=tf.reduce_sum(tf.multiply(tf.to_float(test_onehot),outputs[:,12:]+tf.reshape(heading_bin_centers,(1,6))),axis=1)+tf.constant(np.pi)
    
        return get_box3d_corners_helper(centers,headings,sizes)
    
    def if_false():
        if not flip:
            headings=tf.reduce_sum(tf.multiply(tf.to_float(heading_class_label),outputs[:,12:]+tf.reshape(heading_bin_centers,(1,6))),axis=1)
        else:
            headings=tf.reduce_sum(tf.multiply(tf.to_float(heading_class_label),outputs[:,12:]+tf.reshape(heading_bin_centers,(1,6))),axis=1)+tf.constant(np.pi)
    
 
        return get_box3d_corners_helper(centers,headings,sizes)

    return tf.cond(test,if_true,if_false)


def get_corners(outputs,heading_class_label,stage1_center, flip=False):
    centers=outputs[:,:3]+tf.squeeze(stage1_center,axis=1)
    sizes=outputs[:,3:6]+tf.constant(g_mean_size_arr, tf.float32)
    
    heading_bin_centers = tf.constant(np.arange(0,np.pi,np.pi/NUM_HEADING_BIN), dtype=tf.float32) 
    
    
    
    
    if not flip:
            headings=tf.reduce_sum(tf.multiply(tf.to_float(heading_class_label),outputs[:,12:]+tf.reshape(heading_bin_centers,(1,6))),axis=1)
    else:
            headings=tf.reduce_sum(tf.multiply(tf.to_float(heading_class_label),outputs[:,12:]+tf.reshape(heading_bin_centers,(1,6))),axis=1)+tf.constant(np.pi)
    
 
    return get_box3d_corners_helper(centers,headings,sizes)

    
        
    
    #max_ind = tf.argmax(outputs[6:18], axis=1)
    
    
    #correct_heading_res = tf.multiply(outputs[:,6:18],outputs[:,18:30])
    
    #headings = output[:] + tf.expand_dims(heading_bin_centers[max_ind], 0)
    

(1, 3)


In [12]:
def get_IOU_fn(pred_corners,corners_labels):
    pass
    

In [13]:
tf.reset_default_graph()
with tf.device('/device:GPU:0'):
    num_points_5 =512

    
#     inputs=tf.placeholder(tf.float32,[None,2048,4])#put placeholder

#     idx_5 =tf.placeholder(tf.int32,[None,num_points_5,1])

   
    runfortest=tf.placeholder(tf.bool,name='boooool')
    is_training = tf.placeholder(tf.bool, name='bool')
    lr=tf.placeholder(tf.float32,name='l_rate')

    #label_out_5 = tf.placeholder(tf.int32, [None])
    center_reglabel=tf.placeholder(tf.float32,[None,3], name='center')
    size_reglabel=tf.placeholder(tf.float32,[None,3], name='size')
    heading_class_label=tf.placeholder(tf.int32,[None,6], name='heading')
    heading_reglabel=tf.placeholder(tf.float32,[None,6], name='heading_reg')
    sel_win=tf.placeholder(tf.float32,[None,512,3], name='points')
    corners_labels=tf.placeholder(tf.float32,[None,8,3],name='corners')
#     ret1 = tf.multiply(center_reglabel, 2)
#     ret2 = tf.multiply(size_reglabel,3)
#     ret3 = tf.multiply(heading_class_label,4)
#     ret4 = tf.multiply(sel_win,5)
    
    #sig,logits, label_out,tp,fp,tn,fn, tp_num, maxi,sel_win = classification_net(inputs, label_out_5, idx_5, is_training)
    
#     print('shape of inputs= ',inputs.get_shape())
#     print('shape of inputs= ',idx_5.get_shape())

#     func_fl = focal_loss(tf.squeeze(sig),tf.squeeze(tf.cast(label_out, tf.float32)), gamma=0.75, alpha=0.25)
#     print(label_out.get_shape())

    
#     sel_win=sel_win[:,:,:3]
    sel_win_centroid=tf.reshape(tf.reduce_mean(tf.cast(sel_win,tf.float32),axis=1),(-1,3))
    
    first_center = tf.reduce_mean(tf.norm(sel_win_centroid-center_reglabel, axis=-1))
    
    sel_win_centroid = tf.expand_dims(sel_win_centroid, axis=1)
    
    
    
    print('shape of sel_win_centroid= ',sel_win_centroid.get_shape())
    translated_sel_win=sel_win-sel_win_centroid
    print('shape of translated_sel_win= ',translated_sel_win.get_shape())
    
    center_delta = get_center_regression_net(translated_sel_win)
    center_delta=tf.expand_dims(center_delta, axis=1)
    print('shape of center_delta= ',center_delta.get_shape())
    
    stage1_center = center_delta + sel_win_centroid
    print('shape of stage1_center= ',stage1_center.get_shape())
    
    win_sel_new=sel_win-stage1_center
    print('shape of win_sel_new= ',win_sel_new.get_shape())
    output= get_3d_box_estimation_v1_net(win_sel_new)
    
    print('shape of output= ',output.get_shape())
    
    
    
    #corners_loss = huber_loss(corners_dist, delta=1.0) 
    
#     #get_box3d_corners()
    
    #mask_loss = tf.reduce_mean(func_fl)
    stage1_center_loss= tf.reduce_mean(tf.norm(center_reglabel - \
        tf.squeeze(stage1_center,axis=1), axis=-1))
    center_loss=tf.reduce_mean(tf.norm(center_reglabel - \
        (tf.reshape(output[:,:3],(-1,3))+tf.squeeze(stage1_center,axis=1)), axis=-1))
    size_loss=tf.reduce_mean(tf.norm(size_reglabel - \
        tf.reshape(output[:,3:6],(-1,3)), axis=-1))
    headingclass_loss=tf.reduce_mean( \
        tf.nn.softmax_cross_entropy_with_logits( \
        logits=output[:,6:12], labels=heading_class_label))
    headingclassaccuracy= tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output[:,6:12],axis=-1),tf.argmax(heading_class_label,axis=-1)),tf.float32))
    pred_corners=get_corners2(output,heading_class_label,stage1_center, flip=False,test=runfortest)
    corners_labels=get_corners(tf.concat([center_reglabel, size_reglabel, tf.to_float(heading_class_label), heading_reglabel], axis=1)\
                               , heading_class_label, tf.zeros([5,1,3], dtype=tf.float32), flip=False)
    
    corners_labels_flip = get_corners(tf.concat([center_reglabel, size_reglabel, tf.to_float(heading_class_label), heading_reglabel], axis=1)\
                                      , heading_class_label, tf.zeros([5,1,3], dtype=tf.float32), flip=True)
    
    
    corners_dist = tf.reduce_mean(tf.minimum(tf.norm(pred_corners - corners_labels, axis=-1),
        tf.norm(pred_corners - corners_labels_flip, axis=-1)))
    
    correct_heading_reg_gt = tf.reduce_sum(tf.multiply(heading_reglabel, tf.to_float(heading_class_label)),axis=1)
    correct_headingreg_pred = tf.reduce_sum(tf.multiply(output[:,12:], tf.to_float(heading_class_label)),axis=1)
    headingreg_loss = tf.reduce_mean(tf.norm(correct_heading_reg_gt-correct_headingreg_pred, axis=-1))
    #heading_residual_loss=
    loss=1.0*stage1_center_loss+1.0*center_loss+4.0*size_loss+4.0*headingclass_loss+4.0*headingreg_loss+2.0*corners_dist
    optimizer = tf.train.AdamOptimizer(lr)
    #train_step=optimizer.minimize(loss)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)
    
    

shape of sel_win_centroid=  (?, 1, 3)
shape of translated_sel_win=  (?, 512, 3)
shape of center_delta=  (?, 1, 3)
shape of stage1_center=  (?, 1, 3)
shape of win_sel_new=  (?, 512, 3)
shape of output=  (?, 18)
shape of max_ind=  (?,)
shape of one_hot=  (?, 6)


In [57]:
# class_windows=np.load('./Windows/selwin.npy', allow_pickle=True)
# extras=np.load('./Windows/selextras.npy', allow_pickle=True)

In [76]:
class_windows=np.load('./Windows/val_selwin.npy', allow_pickle=True)
extras=np.load('./Windows/val_selextras.npy', allow_pickle=True)
extras=extras[:-4,:]
class_windows=class_windows[:-4,:]
extras.shape,class_windows.shape

((2235, 9), (2235, 512, 3))

In [77]:
# class_windows.shape,val_class_windows.shape

In [78]:
#class_windows[i:i+batchsize,:,:3].shape

In [79]:
extrascenter=extras[:,2]
extrascenter=np.array([list(x) for x in extrascenter])

In [80]:
# val_extrascenter=val_extras[:,2]
# val_extrascenter=np.array([list(x) for x in val_extrascenter])

In [81]:
# val_extrassize=val_extras[:,6]
# val_extrassize=np.array([list(x) for x in val_extrassize])

In [82]:
extrassize=extras[:,6]
extrassize=np.array([list(x) for x in extrassize])

In [83]:
# val_extrasheading=val_extras[:,3]
# val_extrasheading=np.array([x for x in val_extrasheading])
# val_extrasheading.shape

In [84]:
extrasheading=extras[:,3]
extrasheading=np.array([x for x in extrasheading])
extrasheading.shape

(2235,)

In [85]:
# val_modified_extrasheading=[]
# for x in val_extrasheading:
#     if x<=5:
#         val_modified_extrasheading.append(x)
#     else:
#         val_modified_extrasheading.append(x-6)
# val_extrasheading=np.array(val_modified_extrasheading)

In [86]:
modified_extrasheading=[]
for x in extrasheading:
    if x<=5:
        modified_extrasheading.append(x)
    else:
        modified_extrasheading.append(x-6)
extrasheading=np.array(modified_extrasheading)

In [87]:
# val_extrasheadingl=np.zeros((val_extrasheading.shape[0],6),dtype=np.int32)
# val_extrasheadingl[np.arange(val_extrasheading.shape[0]),val_extrasheading]=1

In [88]:
extrasheadingl=np.zeros((extrasheading.shape[0],6),dtype=np.int32)
extrasheadingl[np.arange(extrasheading.shape[0]),extrasheading]=1

In [89]:
# val_extras_headingres = val_extras[:,4]
# val_extras_headingres = np.reshape(val_extras_headingres,(-1,1))
# print(val_extras_headingres.shape)
# print(val_extras_headingres[0])
# val_extras_headingres = val_extrasheadingl*val_extras_headingres
# print(val_extras_headingres.shape)
# print(val_extras_headingres[0])
# print(val_extrasheadingl[0])

In [90]:
extras_headingres = extras[:,4]
extras_headingres = np.reshape(extras_headingres,(-1,1))
print(extras_headingres.shape)
print(extras_headingres[0])
extras_headingres = extrasheadingl*extras_headingres
print(extras_headingres.shape)
print(extras_headingres[0])
print(extrasheadingl[0])

(2235, 1)
[0.24208863784600743]
(2235, 6)
[0.24208863784600743 0.0 0.0 0.0 0.0 0.0]
[1 0 0 0 0 0]


In [91]:
from box_util import box3d_iou

In [92]:
print(type(sel_win))
print(center_reglabel)
print(size_reglabel)
print(heading_class_label)


<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("center:0", shape=(?, 3), dtype=float32, device=/device:GPU:0)
Tensor("size:0", shape=(?, 3), dtype=float32, device=/device:GPU:0)
Tensor("heading:0", shape=(?, 6), dtype=int32, device=/device:GPU:0)


In [93]:
# print(sel_win.get_shape())
# print(class_windows[i:i+batchsize,:,:3].shape)
# print(center_reglabel.get_shape())
# print(residual_regress_xyzlabels.shape)
# print(size_reglabel.get_shape())
# print(residual_regress_lbhlabels.shape)
# print(heading_class_label.get_shape())
# print(heading_class_l.shape)
# print(is_training.get_shape())

In [94]:
it=0

In [95]:
# print(val_class_windows[it:it+batchsize,:,:].shape)
# print(val_extrascenter[it:it+batchsize,:].shape)
# print(val_extrassize[it:it+batchsize,:].shape)
# print(val_extrasheadingl[it:it+batchsize,:].shape)
# print(val_extras_headingres[it:it+batchsize,:].shape)

In [50]:
it

13403

In [56]:
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:

    sess.run(tf.global_variables_initializer())
   
    
    
    for epoch in range(30):
        ioulist=[]
        tl=[]
        print('**************************************************************************************')
        i=0
        batchsize=5
        pedc=0
        while(i<(extras.shape[0]-5000)):
            pedc+=1
            
            
            out_l =sess.run([train_op,pred_corners,corners_labels,loss, stage1_center_loss, center_loss, size_loss,\
                             headingclass_loss,headingreg_loss,headingclassaccuracy ,corners_dist],\
                            feed_dict={lr: 1e-3/(epoch+1),sel_win:class_windows[i:i+batchsize,:,:3],\
                                       center_reglabel:extrascenter[i:i+batchsize,:],\
                                                            size_reglabel:extrassize[i:i+batchsize,:],\
                                                            heading_class_label:extrasheadingl[i:i+batchsize,:],\
                                                            is_training: True,runfortest:False, heading_reglabel:extras_headingres[i:i+batchsize,:]})
            pred_c=out_l[1]
            label_c=out_l[2]
            
            #print([box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)])
            ioulist+=[box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)]
            
            tl.append(out_l[3:])
            #print(out_l[2:])
            #print('epoch= ',epoch,'pedc= ',pedc,'loss= ',out_l[0],'average loss= ',sum(tl)/len(tl))
            #if pedc%100==0:
            if pedc%100==0:
                save_path = saver.save(sess, "./checkpoints/endtoendday2_"+str(epoch)+'_'+str(pedc)+".ckpt")

                print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print('epoch= ',epoch,'pedc= ',pedc)
                print('average loss= ',np.mean(np.array(tl),axis=0)[0])
                print('stage1_center_loss= ',np.mean(np.array(tl),axis=0)[1])
                print('center_loss= ',np.mean(np.array(tl),axis=0)[2])
                print('size_loss= ',np.mean(np.array(tl),axis=0)[3])
                print('heading_loss= ',np.mean(np.array(tl),axis=0)[4])
                print('heading regression loss= ',np.mean(np.array(tl),axis=0)[5])
                print('heading accuracy= ',np.mean(np.array(tl),axis=0)[6])
                print('corner_dist= ',np.mean(np.array(tl),axis=0)[7])
                print('iou 0.3= ',len(list(filter(lambda x: x>0.3, ioulist)))/len(ioulist))
                print('iou 0.5= ',len(list(filter(lambda x: x>0.5, ioulist)))/len(ioulist))
                tl=[]
                
            i+=batchsize
        it = extras.shape[0]-5000
        
        ioulist_test=[]
        tl_test=[]
        while it<extras.shape[0]:
            pedc+=1
            
            
            out_l =sess.run([pred_corners,corners_labels,loss, stage1_center_loss, center_loss, size_loss,\
                             headingclass_loss,headingreg_loss,headingclassaccuracy ,corners_dist],\
                            feed_dict={lr: 1e-3/(epoch+1),sel_win:class_windows[it:it+batchsize,:,:3],\
                                       center_reglabel:extrascenter[it:it+batchsize,:],\
                                                            size_reglabel:extrassize[it:it+batchsize,:],\
                                                            heading_class_label:extrasheadingl[it:it+batchsize,:],\
                                                            is_training: True,runfortest:True, \
                                       heading_reglabel:extras_headingres[it:it+batchsize,:]})
            pred_c=out_l[0]
            label_c=out_l[1]
            
            #print([box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)])
            ioulist_test+=[box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)]
            
            tl_test.append(out_l[2:])
            #print(out_l[2:])
            #print('epoch= ',epoch,'pedc= ',pedc,'loss= ',out_l[0],'average loss= ',sum(tl)/len(tl))
            #if pedc%100==0:
            if pedc%100==0:
                #save_path = saver.save(sess, "./checkpoints/endtoendday2_"+str(epoch)+'_'+str(pedc)+".ckpt")
                print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&')
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print('epoch= ',epoch,'pedc= ',pedc)
                print('average loss= ',np.mean(np.array(tl_test),axis=0)[0])
                print('stage1_center_loss= ',np.mean(np.array(tl_test),axis=0)[1])
                print('center_loss= ',np.mean(np.array(tl_test),axis=0)[2])
                print('size_loss= ',np.mean(np.array(tl_test),axis=0)[3])
                print('heading_loss= ',np.mean(np.array(tl_test),axis=0)[4])
                print('heading regression loss= ',np.mean(np.array(tl_test),axis=0)[5])
                print('heading accuracy= ',np.mean(np.array(tl_test),axis=0)[6])
                print('corner_dist= ',np.mean(np.array(tl_test),axis=0)[7])
                print('iou 0.3= ',len(list(filter(lambda x: x>0.3, ioulist_test)))/len(ioulist_test))
                print('iou 0.5= ',len(list(filter(lambda x: x>0.5, ioulist_test)))/len(ioulist_test))
                tl_test=[]
            it += batchsize

**************************************************************************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  0 pedc=  100
average loss=  22.297827
stage1_center_loss=  1.9759653
center_loss=  1.1258966
size_loss=  0.7818156
heading_loss=  2.0101724
heading regression loss=  1.4468718
heading accuracy=  0.15999994
corner_dist=  1.1202611
iou 0.3=  0.032
iou 0.5=  0.004
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  0 pedc=  200
average loss=  17.558578
stage1_center_loss=  1.7289512
center_loss=  0.80087286
size_loss=  0.50081193
heading_loss=  1.8871897
heading regression loss=  0.96812147
heading accuracy=  0.22199997
corner_dist=  0.80212873
iou 0.3=  0.046
iou 0.5=  0.005
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  0 pedc=  300
average loss=  16.198202
stage1_center_loss=  1.4852457
center_loss=  0.7560156
size_loss=  0.42252436
heading_loss=  1.8408056
heading regression loss=  0.8532826
heading accuracy=  0.16999996
corn

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  0 pedc=  2500
average loss=  12.321895
stage1_center_loss=  0.57525843
center_loss=  0.27170205
size_loss=  0.32655075
heading_loss=  1.7788113
heading regression loss=  0.5360362
heading accuracy=  0.24599999
corner_dist=  0.45467004
iou 0.3=  0.7
iou 0.5=  0.1813953488372093
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  0 pedc=  2600
average loss=  12.431552
stage1_center_loss=  0.5823091
center_loss=  0.26616952
size_loss=  0.316146
heading_loss=  1.7841994
heading regression loss=  0.5739111
heading accuracy=  0.23199995
corner_dist=  0.44302073
iou 0.3=  0.7030188679245283
iou 0.5=  0.1811320754716981
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  0 pedc=  2700
average loss=  12.3855

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  1 pedc=  1800
average loss=  10.932361
stage1_center_loss=  0.35068816
center_loss=  0.21014413
size_loss=  0.29629487
heading_loss=  1.7629602
heading regression loss=  0.39626256
heading accuracy=  0.28000003
corner_dist=  0.27472916
iou 0.3=  0.8453333333333334
iou 0.5=  0.3948888888888889
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  1 pedc=  1900
average loss=  11.069847
stage1_center_loss=  0.34734944
center_loss=  0.24358633
size_loss=  0.29402485
heading_loss=  1.7727196
heading regression loss=  0.4025389
heading accuracy=  0.27199998
corner_dist=  0.30088997
iou 0.3=  0.844
iou 0.5=  0.39294736842105266
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  1 pedc=  2000
average loss=  11.436593
stage1_center_loss=  0.5852382
center_loss=  0.2580541
size_loss=  0.28410888
heading_loss=  1.8023778
heading regression loss=  0.4059278
heading accuracy=  0.19399996
corner_dist=  0.31182092
iou 0.3=  0.8438
i

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  2 pedc=  1000
average loss=  10.876523
stage1_center_loss=  0.40672988
center_loss=  0.2552259
size_loss=  0.2677085
heading_loss=  1.759123
heading regression loss=  0.37554184
heading accuracy=  0.2519999
corner_dist=  0.3025369
iou 0.3=  0.8472
iou 0.5=  0.413
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  2 pedc=  1100
average loss=  10.691455
stage1_center_loss=  0.35213867
center_loss=  0.23739523
size_loss=  0.27636713
heading_loss=  1.7502983
heading regression loss=  0.35458413
heading accuracy=  0.23199996
corner_dist=  0.28846043
iou 0.3=  0.8458181818181818
iou 0.5=  0.41345454545454546
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  2 pedc=  1200
average loss=  10.927864
stage1_center_loss=  0.40797114
center_loss=  0.24552135
size_loss=  0.27532223
heading_loss=  1.7713755
heading regression loss=  0.37296218
heading accuracy=  0.23999998
corner_dist=  0.29786414
iou 0.3=  0.844
iou 0.5=  0.408

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  3 pedc=  200
average loss=  10.415058
stage1_center_loss=  0.32766548
center_loss=  0.2000411
size_loss=  0.26730877
heading_loss=  1.7443669
heading regression loss=  0.3320178
heading accuracy=  0.274
corner_dist=  0.25628734
iou 0.3=  0.889
iou 0.5=  0.485
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  3 pedc=  300
average loss=  10.435568
stage1_center_loss=  0.3383473
center_loss=  0.21117231
size_loss=  0.27244857
heading_loss=  1.7122014
heading regression loss=  0.3530622
heading accuracy=  0.28799996
corner_dist=  0.26759928
iou 0.3=  0.8886666666666667
iou 0.5=  0.476
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  3 pedc=  400
average loss=  10.282365
stage1_center_loss=  0.34161118
center_loss=  0.19682974
size_loss=  0.28221256
heading_loss=  1.688499
heading regression loss=  0.33667347
heading accuracy=  0.294
corner_dist=  0.25719166
iou 0.3=  0.896
iou 0.5=  0.4855
%%%%%%%%%%%%%%%%%%%%%%%%%%

...

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  21 pedc=  1000
average loss=  8.041944
stage1_center_loss=  0.17800406
center_loss=  0.15578544
size_loss=  0.23610665
heading_loss=  1.3164763
heading regression loss=  0.2691752
heading accuracy=  0.49799985
corner_dist=  0.21056058
iou 0.3=  0.9592
iou 0.5=  0.684
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  21 pedc=  1100
average loss=  8.102784
stage1_center_loss=  0.18197714
center_loss=  0.16454306
size_loss=  0.2543801
heading_loss=  1.3383359
heading regression loss=  0.23694366
heading accuracy=  0.47799996
corner_dist=  0.21881229
iou 0.3=  0.9569090909090909
iou 0.5=  0.6825454545454546
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  21 pedc=  1200
average loss=  7.849962
stage1_center_loss=  0.1847847
center_loss=  0.16941114
size_loss=  0.24051784
heading_loss=  1.2745216
heading regression loss=  0.24784592
heading accuracy=  0.46000013
corner_dist=  0.22211239
iou 0.3=  0.9556666666666667
i

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  22 pedc=  200
average loss=  8.137501
stage1_center_loss=  0.17910326
center_loss=  0.15640713
size_loss=  0.25063512
heading_loss=  1.3301287
heading regression loss=  0.26312208
heading accuracy=  0.48999995
corner_dist=  0.21322359
iou 0.3=  0.958
iou 0.5=  0.694
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  22 pedc=  300
average loss=  7.964648
stage1_center_loss=  0.1901516
center_loss=  0.1570385
size_loss=  0.24890475
heading_loss=  1.2775428
heading regression loss=  0.27039245
heading accuracy=  0.52799994
corner_dist=  0.21504779
iou 0.3=  0.9606666666666667
iou 0.5=  0.6846666666666666
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  22 pedc=  400
average loss=  7.9952083
stage1_center_loss=  0.17288941
center_loss=  0.14917716
size_loss=  0.2520242
heading_loss=  1.2970376
heading regression loss=  0.26490748
heading accuracy=  0.5040001
corner_dist=  0.20863253
iou 0.3=  0.96
iou 0.5=  0.6935
%%

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  22 pedc=  2600
average loss=  8.508274
stage1_center_loss=  0.16783392
center_loss=  0.15413252
size_loss=  0.2592755
heading_loss=  1.3476434
heading regression loss=  0.29079908
heading accuracy=  0.48600003
corner_dist=  0.29771855
iou 0.3=  0.9569811320754718
iou 0.5=  0.6335849056603774
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  22 pedc=  2700
average loss=  9.246935
stage1_center_loss=  0.17452443
center_loss=  0.16241984
size_loss=  0.26287892
heading_loss=  1.4843122
heading regression loss=  0.31601787
heading accuracy=  0.41600013
corner_dist=  0.3285776
iou 0.3=  0.9536507936507936
iou 0.5=  0.626031746031746
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  22 pedc=  2800
aver

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  23 pedc=  1900
average loss=  7.622493
stage1_center_loss=  0.1811356
center_loss=  0.15237018
size_loss=  0.25428408
heading_loss=  1.2110982
heading regression loss=  0.2518393
heading accuracy=  0.542
corner_dist=  0.21005058
iou 0.3=  0.9615789473684211
iou 0.5=  0.7003157894736842
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  23 pedc=  2000
average loss=  7.771027
stage1_center_loss=  0.18833026
center_loss=  0.16308092
size_loss=  0.24168144
heading_loss=  1.255941
heading regression loss=  0.24941136
heading accuracy=  0.5079999
corner_dist=  0.21574032
iou 0.3=  0.9615
iou 0.5=  0.7006
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  23 pedc=  2100
average loss=  8.396869
stage1_center_loss=  0.16789931
center_loss=  0.1481975
size_loss=  0.2617044
heading_loss=  1.2824999
heading regression loss=  0.33759946
heading accuracy= 

KeyboardInterrupt: 

In [ ]:
# TEST SET EVALUATION

In [97]:
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:

    sess.run(tf.global_variables_initializer())
   
    
    saver.restore(sess, "./checkpoints/endtoendday2_23_2000.ckpt")
    for epoch in range(1):
        ioulist=[]
        tl=[]
        print('**************************************************************************************')
        i=0
        batchsize=5
        pedc=0
        while(i<(extras.shape[0])):
            pedc+=1
            
            
            out_l =sess.run([pred_corners,corners_labels,loss, stage1_center_loss, center_loss, size_loss,\
                             headingclass_loss,headingreg_loss,headingclassaccuracy ,corners_dist],\
                            feed_dict={lr: 1e-3/(epoch+1),sel_win:class_windows[i:i+batchsize,:,:3],\
                                       center_reglabel:extrascenter[i:i+batchsize,:],\
                                                            size_reglabel:extrassize[i:i+batchsize,:],\
                                                            heading_class_label:extrasheadingl[i:i+batchsize,:],\
                                                            is_training: True,runfortest:False, heading_reglabel:extras_headingres[i:i+batchsize,:]})
            pred_c=out_l[0]
            label_c=out_l[1]
            
            #print([box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)])
            ioulist+=[box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)]
            
            tl.append(out_l[2:])
            #print(out_l[2:])
            #print('epoch= ',epoch,'pedc= ',pedc,'loss= ',out_l[0],'average loss= ',sum(tl)/len(tl))
            #if pedc%100==0:
            #if pedc%100==0:
                #save_path = saver.save(sess, "./checkpoints/endtoendday2_"+str(epoch)+'_'+str(pedc)+".ckpt"
                
            i+=batchsize
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print('epoch= ',epoch,'pedc= ',pedc)
        print('average loss= ',np.mean(np.array(tl),axis=0)[0])
        print('stage1_center_loss= ',np.mean(np.array(tl),axis=0)[1])
        print('center_loss= ',np.mean(np.array(tl),axis=0)[2])
        print('size_loss= ',np.mean(np.array(tl),axis=0)[3])
        print('heading_loss= ',np.mean(np.array(tl),axis=0)[4])
        print('heading regression loss= ',np.mean(np.array(tl),axis=0)[5])
        print('heading accuracy= ',np.mean(np.array(tl),axis=0)[6])
        print('corner_dist= ',np.mean(np.array(tl),axis=0)[7])
        print('iou 0.3= ',len(list(filter(lambda x: x>0.3, ioulist)))/len(ioulist))
        print('iou 0.5= ',len(list(filter(lambda x: x>0.5, ioulist)))/len(ioulist))
        print('iou 0.7= ',len(list(filter(lambda x: x>0.7, ioulist)))/len(ioulist))

INFO:tensorflow:Restoring parameters from ./checkpoints/endtoendday2_23_2000.ckpt
**************************************************************************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
epoch=  0 pedc=  447
average loss=  11.190585
stage1_center_loss=  0.5500363
center_loss=  0.529746
size_loss=  0.27213696
heading_loss=  1.5987085
heading regression loss=  0.37164584
heading accuracy=  0.35525692
corner_dist=  0.57042027
iou 0.3=  0.8496644295302014
iou 0.5=  0.5955257270693513
iou 0.7=  0.09082774049217002


In [49]:
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:

    sess.run(tf.global_variables_initializer())
   
    
    saver.restore(sess, "./checkpoints/model_new_labels_pointwiseIOU_5_10000.ckpt")
    for epoch in range(30):
        ioulist=[]
        tl=[]
        print('**************************************************************************************')
        i=0
        batchsize=5
        pedc=0
        while(i<(extras.shape[0])):
            pedc+=1
            
            
            out_l =sess.run([train_op,pred_corners,corners_labels,loss, stage1_center_loss, center_loss, size_loss,\
                             headingclass_loss,headingreg_loss,headingclassaccuracy ,corners_dist],\
                            feed_dict={lr: 1e-3/(epoch+1),sel_win:class_windows[i:i+batchsize,:,:3],\
                                       center_reglabel:extrascenter[i:i+batchsize,:],\
                                                            size_reglabel:extrassize[i:i+batchsize,:],\
                                                            heading_class_label:extrasheadingl[i:i+batchsize,:],\
                                                            is_training: True,runfortest:False, heading_reglabel:extras_headingres[i:i+batchsize,:]})
            pred_c=out_l[1]
            label_c=out_l[2]
            
            #print([box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)])
            ioulist+=[box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)]
            
            tl.append(out_l[3:])
            #print(out_l[2:])
            #print('epoch= ',epoch,'pedc= ',pedc,'loss= ',out_l[0],'average loss= ',sum(tl)/len(tl))
            #if pedc%100==0:
            if pedc%100==0:
                save_path = saver.save(sess, "./checkpoints/endtoendday2_"+str(epoch)+'_'+str(pedc)+".ckpt")

                print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print('epoch= ',epoch,'pedc= ',pedc)
                print('average loss= ',np.mean(np.array(tl),axis=0)[0])
                print('stage1_center_loss= ',np.mean(np.array(tl),axis=0)[1])
                print('center_loss= ',np.mean(np.array(tl),axis=0)[2])
                print('size_loss= ',np.mean(np.array(tl),axis=0)[3])
                print('heading_loss= ',np.mean(np.array(tl),axis=0)[4])
                print('heading regression loss= ',np.mean(np.array(tl),axis=0)[5])
                print('heading accuracy= ',np.mean(np.array(tl),axis=0)[6])
                print('corner_dist= ',np.mean(np.array(tl),axis=0)[7])
                print('iou 0.3= ',len(list(filter(lambda x: x>0.3, ioulist)))/len(ioulist))
                print('iou 0.5= ',len(list(filter(lambda x: x>0.5, ioulist)))/len(ioulist))
                tl=[]
                
            i+=batchsize
        it = 0
        
        ioulist_test=[]
        tl_test=[]
        while it<val_extras.shape[0]:
            pedc+=1
            
            
            out_l =sess.run([pred_corners,corners_labels,loss, stage1_center_loss, center_loss, size_loss,\
                             headingclass_loss,headingreg_loss,headingclassaccuracy ,corners_dist],\
                            feed_dict={lr: 1e-3/(epoch+1),sel_win:val_class_windows[it:it+batchsize,:,:3],\
                                       center_reglabel:val_extrascenter[it:it+batchsize,:],\
                                                            size_reglabel:val_extrassize[it:it+batchsize,:],\
                                                            heading_class_label:val_extrasheadingl[it:it+batchsize,:],\
                                                            is_training: True,runfortest:True, \
                                       heading_reglabel:val_extras_headingres[it:it+batchsize,:]})
            pred_c=out_l[0]
            label_c=out_l[1]
            
            #print([box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)])
            ioulist_test+=[box3d_iou(pred_c[x],label_c[x]) for x in range(batchsize)]
            
            tl_test.append(out_l[2:])
            #print(out_l[2:])
            #print('epoch= ',epoch,'pedc= ',pedc,'loss= ',out_l[0],'average loss= ',sum(tl)/len(tl))
            #if pedc%100==0:
            if pedc%100==0:
                #save_path = saver.save(sess, "./checkpoints/endtoendday2_"+str(epoch)+'_'+str(pedc)+".ckpt")
                print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& TEST RESULTS&&&&&&&&&&&&&&&&&&&&&&&&&&')
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print('epoch= ',epoch,'pedc= ',pedc)
                print('average loss= ',np.mean(np.array(tl_test),axis=0)[0])
                print('stage1_center_loss= ',np.mean(np.array(tl_test),axis=0)[1])
                print('center_loss= ',np.mean(np.array(tl_test),axis=0)[2])
                print('size_loss= ',np.mean(np.array(tl_test),axis=0)[3])
                print('heading_loss= ',np.mean(np.array(tl_test),axis=0)[4])
                print('heading regression loss= ',np.mean(np.array(tl_test),axis=0)[5])
                print('heading accuracy= ',np.mean(np.array(tl_test),axis=0)[6])
                print('corner_dist= ',np.mean(np.array(tl_test),axis=0)[7])
                print('iou 0.3= ',len(list(filter(lambda x: x>0.3, ioulist_test)))/len(ioulist_test))
                print('iou 0.5= ',len(list(filter(lambda x: x>0.5, ioulist_test)))/len(ioulist_test))
                tl_test=[]
            it += batchsize

13403

In [ ]:
type(residual_regress_xyzlabels),type(residual_regress_lbhlabels),type(heading_class_num)

In [ ]:
# voxels_train_np=[i.get_feature_vector() for i in voxels_train]

In [ ]:
obj_voxels=[]
#obj_centroids=[]
#obj_centroids_unique=[]
#obj_center_label=[]
#obj_corners=[]
obj_lbh=[]
obj_extras=[]
#obj_extras=[]
#obj_heading=[]
for i in range(len(voxels_train)):
    try :
        obj_voxels.append(voxels_train[i].get_feature_vector(mode='density'))
        #obj_centroids.append(centroids[i])
        #obj_centroids_unique.append(centroids_unique[i])
        #obj_center_label.append(center_label[i])
        #obj_corners.append(corners[i])
        obj_lbh.append(lbh[i])
        obj_extras.append(extras[i])
        #obj_extras.append(extras[i])
        #obj_heading.append(heading[i])
    except IndexError:
        pass
    except ValueError:
        pass

In [ ]:
np.save('./Windows/voxels_denstrue2',np.array(obj_voxels), allow_pickle=True)

np.save('./Windows/lbh_denstrue2',np.array(obj_lbh) , allow_pickle=True)

In [ ]:

lbh=np.load('./Windows/lbh_jtrue2.npy', allow_pickle=True)
voxels_train=np.load('./Windows/voxels_jtrue2.npy', allow_pickle=True)

In [ ]:
del voxels_train
del lbh

In [ ]:
len(obj_voxels),len(obj_centroids),len(obj_centroids_unique),len(obj_center_label),len(obj_corners),len(obj_lbh),len(obj_extras),len(obj_heading)

In [ ]:
#check the distances

import matplotlib.pyplot as plt
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1)))
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1)))
print(np.mean(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1))))
plt.hist(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1)), bins=100)

In [ ]:
#check the distances

import matplotlib.pyplot as plt
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1)))
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1)))
print(np.mean(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1))))
plt.hist(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1)), bins=20)

In [ ]:
# for i in range(len(voxels_train)):
#     if i not in [289,315,431,728,746,1283,1543,1865,2128,3485,3753,4429,4928,5048,5432,5907,6191,6999,7100,7574,\
#                 7687,9404,10179]:
#         voxels_train[i].get_feature_vector()

In [ ]:
# obj_voxels = []
# center_label = []
# centroids= []
# print(np.array(point_cld[0]).shape)
# for i in range(len(voxels_train)):
#     if i not in [289,315,431,728,746,1283,1543,1865,2128,3485,3753,4429,4928,5048,5432,5907,6191,6999,7100,7574,\
#                 7687,9404,10179]:
#         obj_voxels.append(voxels_train[i].get_feature_vector())
#         #center_label.append(TRAIN_DATASET.get_center_view_box3d_center(trix[i]))
#         #centroids.append(np.mean(np.unique(np.array(point_cld[i]),axis=0), axis=0))

In [ ]:
obj_voxels_modified = []
#center_label_modified = []
#centroids_modified = []
#obj_corners_modified=[]
obj_lbh_modified=[]
#obj_extras_modified=[]
#obj_heading_modified=[]

for i in range(len(obj_voxels)):
    dist = np.sqrt(np.sum((np.array(obj_center_label[i])-np.array(obj_centroids)[i:i+1,:3])**2, axis=1))
    if dist<1:
        #print(dist)
        obj_voxels_modified.append(obj_voxels[i])
        center_label_modified.append(obj_center_label[i])
        centroids_modified.append(obj_centroids[i])
        obj_corners_modified.append(obj_corners[i])
        obj_lbh_modified.append(obj_lbh[i])
        obj_extras_modified.append(obj_extras[i])
        obj_heading_modified.append(obj_heading[i])

In [ ]:
len(obj_voxels_modified),len(center_label_modified),len(centroids_modified),len(obj_corners_modified),len(obj_lbh_modified),len(obj_extras_modified),len(obj_heading_modified)

In [ ]:
max_x,max_y,max_z=0,0,0
for v in obj_voxels:
    max_x=max(max_x,v.shape[0])
    max_y=max(max_y,v.shape[1])
    max_z=max(max_z,v.shape[2])
print( max_x,max_y,max_z)

In [ ]:
pad_shape=[30,45,20]
padded_voxels=[]
ccc=0
for v in obj_voxels:
    ccc+=1
    print(ccc)
    dx=pad_shape[0]-v.shape[0]
    dy=pad_shape[1]-v.shape[1]
    dz=pad_shape[2]-v.shape[2]
    npad=((dx//2,dx-dx//2),(dy//2,dy-dy//2),(dz//2,dz-dz//2))
    padded_voxels.append(np.pad(v,pad_width=npad,mode='constant',constant_values=0))

In [ ]:
np.save('./Windows/padded_voxels_densetrue',np.array(padded_voxels), allow_pickle=True)
np.save('./Windows/fulllbh_densetrue',np.array(obj_lbh) , allow_pickle=True)
np.save('./Windows/extras_densetrue',np.array(obj_extras) , allow_pickle=True)



In [ ]:
v=np.load('./Windows/lbh_densetrue.npy', allow_pickle=True)
v=np.load('./Windows/padded_voxels_densetrue.npy', allow_pickle=True)
v=np.load('./Windows/extras_densetrue.npy', allow_pickle=True)

In [ ]:
obj_corners_modified[0]

In [ ]:
obj_corners_modified[0][:,1].max()

In [ ]:
list(np.unique(obj_corners_modified[0][:,1],1)[1])

In [ ]:
testi=[]
for i in range(100):
    if TRAIN_DATASET.__getitem__(trix[i])[-1][1]:
        testi.append(i) 

In [ ]:
TRAIN_DATASET.__getitem__(trix[4])[5:7]

In [ ]:
for i in testi:
    print(provider.class2size(TRAIN_DATASET.__getitem__(trix[i])[5],TRAIN_DATASET.__getitem__(trix[i])[6]))

In [ ]:
TRAIN_DATASET.get_center_view_box3d(trix[4])

In [ ]:
obj_corners_modified[0]

In [ ]:
for c in obj_corners_modified[:len(testi)]:
    print(finddim(c))

In [ ]:
def finddim(corners):
    c=np.copy(corners)
    height=c[:,1].max()-c[:,1].min()
    fourp=np.unique(np.delete(c,1,1),axis=0)
    point1=fourp[0]
    d=[]
    for point2 in fourp[1:]:
        d.append(np.linalg.norm(point1-point2))
    d=sorted(d)
    breadth=d[0]
    length=d[1]
    return np.array([length,breadth,height])
lbh=[]
for c in obj_corners_modified:
    lbh.append(finddim(c))

In [ ]:
residual_centers=[]

for i in range(len(centroids_modified)):
    residual_centers.append(center_label_modified[i]-np.array(centroids_modified)[i,:3])
    

In [ ]:
for residue in residual_centers:
    if np.sqrt(np.sum(residue**2))>1:
        print('ERROR')

In [ ]:
np.save('./Windows/centers',center_label_modified, allow_pickle=True)
np.save('./Windows/centroids',centroids_modified , allow_pickle=True)
np.save('./Windows/corners',obj_corners_modified, allow_pickle=True)
np.save('./Windows/lbh',obj_lbh_modified, allow_pickle=True)
np.save('./Windows/extras',obj_extras_modified, allow_pickle=True)
np.save('./Windows/heading_angle',obj_heading_modified, allow_pickle=True)
np.save('./Windows/res_centerlabels', residual_centers, allow_pickle=True)
np.save('./Windows/voxels', padded_voxels, allow_pickle=True)

In [ ]:
plt.hist(rdist,5)

In [ ]:
# np.save('./Windows/voxels', padded_voxels, allow_pickle=True)
# np.save('./Windows/centerlabels', residual_centers, allow_pickle=True)
# np.save('./Windows/lbh', lbh, allow_pickle=True)
# np.save('./Windows/corners', obj_corners_modified, allow_pickle=True)

In [ ]:
for v in padded_voxels:
    print(v.shape)

In [ ]:
#np.linalg.norm(np.array(center_label[:1]), np.array(centroids)[:1,:3], axis=1)
print(center_label[:1])
print(np.array(centroids)[:1,:3])
print(np.sum((np.array(center_label[:1])-np.array(centroids)[:1,:3])**2))

In [ ]:
a,b,c,d,e,f,g,h,i = TRAIN_DATASET.__getitem__(100)
print(c)
print(TRAIN_DATASET.get_center_view_box3d_center(100))

In [ ]:
np.unique(point_cld[i],axis=0)

In [ ]:
rnge= []
for i in range(len(point_cld)):
    rnge.append(np.max(np.array(point_cld[i]), axis=0)-np.min(np.array(point_cld[i]), axis=0))
print(len(rnge))

In [ ]:
np.max(np.array(rnge), axis=0)

In [ ]:
np.min(np.array(point_cld[0])[:,2])

In [ ]:
np.max(np.array(point_cld[0]), axis=0)

In [ ]:
outliers = []
for i in range(len(point_cld)):
    o = np.max(np.array(point_cld[i])[:,2], axis=0)-np.min(np.array(point_cld[i])[:,2], axis=0)
    if o>1.0:
        outliers.append([i,o])
print(len(outliers))

In [ ]:
ex = 10
print(voxels_train[ex].get_feature_vector().shape)
plotFromVoxels(voxels_train[ex].get_feature_vector())

In [ ]:
j= 0
for i in range(1000):
    #print(TRAIN_DATASET.type_list[trix[i]])
    if TRAIN_DATASET.type_list[trix[i]]=='Pedestrian':
        if j==60:
            print(j)
            print(i)
            seg,a,b,c,d,e,f,g,h = TRAIN_DATASET[60]     
            seg = pd.DataFrame(seg[:,:3])
            seg.columns = ['x','y','z']
            cloud = PyntCloud(seg)
            cloud.plot()
                
            
        j += 1
ex  = 60
seg,a,b,c,d,e,f,g,h = TRAIN_DATASET[trix[386]]
print(seg.shape)                    
seg = pd.DataFrame(seg[:,:3])
seg.columns = ['x','y','z']
cloud = PyntCloud(seg)
cloud.plot()

In [ ]:
import sys
import os

import scipy.ndimage as nd
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
import skimage.measure as sk

from mpl_toolkits import mplot3d

import trimesh
from stl import mesh


In [ ]:
def plotFromVF(vertices, faces):
    input_vec = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            input_vec.vectors[i][j] = vertices[f[j],:]
    figure = plt.figure()
    axes = mplot3d.Axes3D(figure)
    axes.add_collection3d(mplot3d.art3d.Poly3DCollection(input_vec.vectors))
    scale = input_vec.points.flatten(-1)
    axes.auto_scale_xyz(scale, scale, scale)
    plt.show()

def plotFromVoxels(voxels):
    #print(voxels.nonzero())
    x,y,z = voxels.nonzero()
    fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    #ax.scatter( 0.05*x, z*0.05, -0.1*y, zdir='z', c= 'red', norm=0.05)
    ax = fig.gca(projection='3d')
    ax.voxels(0.05*voxels)
    plt.show()

def getVFByMarchingCubes(voxels, threshold=0.5):
    #print(len(sk.marching_cubes(voxels, level=threshold)))
    v, f, n, m =  sk.marching_cubes(voxels, level=threshold)
    return v, f

def plotMeshFromVoxels(voxels, threshold=0.5):
    #print(getVFByMarchingCubes(voxels, threshold))
    v,f = getVFByMarchingCubes(voxels, threshold)
    plotFromVF(v,f)

def plotVoxelVisdom(voxels, visdom, title):
    v, f = getVFByMarchingCubes(voxels)
    visdom.mesh(X=v, Y=f, opts=dict(opacity=0.5, title=title))

In [ ]:
#plotFromVoxels(np.array([[[0,1,0],[0,0,0],[0,1,1]],[[1,1,1],[1,1,1],[0,1,1]]]))
plotMeshFromVoxels(np.array([[[0,1,0],[0,0,0],[0,1,1]],[[1,1,1],[1,1,1],[0,1,1]]]))

In [ ]:
voxelgrid_id = cloud.add_structure("voxelgrid", size_x=0.05, size_y=0.05, size_z=0.05)

In [ ]:
seg['z'].max()-seg['z'].min()

In [ ]:
voxelgrid = cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.get_feature_vector().shape

In [ ]:
seg.drop_duplicates()

In [ ]:
cloud.plot()

In [ ]:
seg.isnull().any()

In [ ]:
def pad_trunc_voxels():
    for voxel in voxels_train:
        dim=voxel.shape
        temp=np.zeros((30,40,20))
        temp
            

In [ ]:
print(voxelgrid.get_feature_vector().shape)
print(np.sum(voxelgrid.get_feature_vector()))
print(voxelgrid.get_feature_vector())

In [ ]:
voxelgrid.plot()

In [ ]:
cloud.plot()

In [ ]:
import datetime
import logging
from random import shuffle
#logging.basicConfig(filename='run_1.log',level=logging.DEBUG)
from tensorflow.python import debug as tf_debug
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))

IOU_list = []
saver = tf.train.Saver()
trix=[cc for cc in range(TRAIN_DATASET.__len__())]
shuffle(trix)
prob = []
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    #sess=tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.run(tf.global_variables_initializer())
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
    # uncomment this when we want to load the checkpoint model
    saver.restore(sess, "./checkpoints/model_new_labels_pointwiseIOU_5_10000.ckpt")

    
    for j in range(6,15):   
        total_tp=0
        total_fp=0
        total_tn=0
        total_fn=0
        total_tpnums = 0
        total_fpnums = 0
        total_fnnums = 0
        
        
        print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
        
        print('Epoch Number........................................',j)
        pedc=0
        
        for i in trix:
        #todo reformat code to have lists instead of many variable
            if TRAIN_DATASET.type_list[i]=='Pedestrian':
                pedc+=1
                if pedc%500==0:
                    total_tp=0
                    total_fp=0
                    total_tn=0
                    total_fn=0
                    total_tpnums = 0
                    total_fpnums = 0
                    total_fnnums = 0
                point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(i)

                index = np.argsort(point_set[:,2])
                point_set = point_set[index]
                labels = labels[index]

                corners_gt = TRAIN_DATASET.get_center_view_box3d(i)
                
 
                s5,e5,l = indices(1.0,0.1,point_set,labels, corners_gt)
        
                #l1 = (l1>=0.6)
                
                print('Label shape', l.shape)


                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)



                    point_set = np.expand_dims(point_set, axis=0)
                        
                    _, x,tpos,fpos,tneg,fneg, tpnums, maxind =sess.run([train_op, loss,tp,fp,tn,fn, tp_num, maxi],feed_dict={idx_5:ind_5,\
                                                               label_out_5: l.astype(np.int32),\
                                                               inputs:point_set, is_training: True})
                    total_tp+=tpos
                    total_fp+=fpos
                    total_tn+=tneg
                    total_fn+=fneg
                    total_tpnums+= tpnums
                    total_fpnums+= (1-tpnums)
                    total_fnnums+= (1-tpnums)
                    ind_5[maxind]
                    print('----------------------------------------------------------------------------------')
                    print('Epoch ', j , 'Loss for the current example ',i,' = ', x,'pedc = ',pedc)
                    #logging.info('Loss for the current example ',i,' = ', x)
                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                    print('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                          ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    #logging.info('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                    #      ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    print('Single metrics: Precision = ',(total_tpnums*1.0)/(total_tpnums+total_fpnums),\
                          ' Recall = ',(total_tpnums*1.0)/(total_tpnums+total_fnnums))
                    print('----------------------------------------------------------------------------------')
                    
                    if(pedc%5000==0):
                        save_path = saver.save(sess, "./checkpoints/model_new_labels_pointwiseIOU_"+str(j)+'_'+str(pedc)+".ckpt")
                    

                #print(c)
                #print(n)
            #i += 1
        #-----------------------------------------------------------------------------------------------#
        print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
        k = 0
        Vtotal_tp=0
        Vtotal_fp=0
        Vtotal_tn=0
        Vtotal_fn=0
        Vtotal_tpnums = 0
        Vtotal_fpnums = 0
        Vtotal_fnnums = 0
        while k<len(VAL_DATASET):
        #todo reformat code to have lists instead of many variable

            if VAL_DATASET.type_list[k]=='Pedestrian':

                point_set,labels,a,b,c,d,e,f,cls = VAL_DATASET.__getitem__(k)

                index = np.argsort(point_set[:,2])
                point_set = point_set[index]
                labels = labels[index]

                
                corners_gt = VAL_DATASET.get_center_view_box3d(k)
                
                #print(point_set.shape)

                s5,e5,l = indices(1.0,0.1,point_set,labels, corners_gt)
                
                #l1 = l1>0.5

#                 s1, e1, l1 = indices(k, 1.1, 0.2, 'val')

#                 s2, e2, l2 = indices(k, 1.3, 0.2, 'val')

#                 s3, e3, l3 = indices(k, 0.7, 0.1, 'val')

#                 s4, e4, l4 = indices(k, 0.5, 0.1, 'val')

                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)

#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]

                    point_set = np.expand_dims(point_set, axis=0)
                    print(Vtotal_tp)
                    x,tpos,fpos,tneg,fneg, tp_nums, maxind =sess.run([loss,tp,fp,tn,fn, tp_num, maxi],feed_dict={idx_5:ind_5, \
                                                               label_out_5: l.astype(np.int32), inputs:point_set, is_training: True})


                    #window_points = ind_5[maxind]

                    #window_points = point_set[0,ind_5[maxind].reshape((-1))][:,:3]
                    #min_x, min_y, min_z = np.min(window_points,axis=0)
                    #max_x, max_y, max_z = np.max(window_points,axis=0)
                    #corners = [[min_x,max_y,max_z],[min_x,max_y,min_z],[max_x,max_y,min_z],[max_x,max_y,max_z],\
                    #           [min_x,min_y,max_z],[min_x,min_y,min_z],[max_x,min_y,min_z],[max_x,min_y,max_z]]
                    #corners=np.array(corners)
                    #corners_gt = VAL_DATASET.get_center_view_box3d(k)
                    #IOU, IOU_2D, inter, inter_vol = box_util.box3d_iou(corners, corners_gt)
                    #print('IOUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU')
                    #print(IOU, IOU_2D, inter, inter_vol)
                    #IOU_list.append([IOU, IOU_2D, inter, inter_vol, k])
                    #if inter>0.4:
                    #    c += 1
                    #    if inter>0.5:
                    #        c1 += 1
                    #if inter_vol>0.4:
                    #    c2 += 1
                    #    if inter_vol>0.5:
                    #        c3 +=1 

                    print(Vtotal_tp)
                    Vtotal_tp +=tpos
                    Vtotal_fp +=fpos
                    Vtotal_tn +=tneg
                    Vtotal_fn +=fneg
                    Vtotal_tpnums+= tp_nums
                    Vtotal_fpnums+= (1-tp_nums)
                    Vtotal_fnnums+= (1-tp_nums)
                    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
                    print('Epoch ', j , 'Loss for the current example ',k,' = ', x)
                    #logging.info('Loss for the current example ',i,' = ', x)
                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                    print('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                          ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))

                    print('Single metrics: Precision = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fpnums),\
                          ' Recall = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fnnums))
                    #logging.info('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                    #      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

            k += 1

        #----------------------------------------------------------------------------------------------#
#         prob = sess.run([sig],feed_dict={idx_5:ind_5, idx_1:ind_1, idx_2:ind_2, idx_3:ind_3, idx_4:ind_4,\
#                                                        label_out_5: l5.astype(np.int32), label_out_1: l1.astype(np.int32),\
#                                                        label_out_2: l2.astype(np.int32), label_out_3: l3.astype(np.int32),\
#                                                        label_out_4: l4.astype(np.int32), inputs:point_set,  is_training: False})
#         print(prob)
        print(ind_5)
        print(label_out_5)
        prob = sess.run([sig],feed_dict={idx_5:ind_5,  \
                                                       label_out_5: l.astype(np.int32), inputs:point_set,  is_training: True})
        print(prob)
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S") )

In [ ]:
n = np.random.rand(31,26,12)
padded_n=np.zeros((40,30,20))
padded_n[:min(n.shape[0],40),:min(n.shape[1],30),:min(n.shape[2],20)]=n


In [ ]:
inp=tf.placeholder(tf.float32,shhape=(1,30,45,20))

conv3d1=tf_util.conv3d(inp,num_output_channels=64,kernel_size=[3,3,3],scope='convr1', is_training = True)
conv3d2=tf_util.conv3d(conv3d1,num_output_channels=128,kernel_size=[3,3,3],scope='convr2', is_training = True)
conv3d3=tf_util.conv3d(conv3d2,num_output_channels=256,kernel_size=[3,3,3],scope='convr3', is_training = True)

dense=conv2d(conv3d2,num_output_channels=30,kernel_size=[40,40,30],scope='dense1r', is_training = True)
out=conv2d(dense,num_output_channels=2,kernel_size=[1,1,30],scope='dense2r', is_training = True)

In [ ]:
import pyntcloud
import pickle
import tensorflow as tf
import numpy as np
import importlib
from train import provider
from train import box_util

In [ ]:
import tensorflow as tf
import numpy as np
from model_util import huber_loss

In [ ]:
from model import tf_util

In [ ]:
centers=np.load('./Windows/centers.npy', allow_pickle=True)
centroids=np.load('./Windows/centroids.npy', allow_pickle=True)
corners=np.load('./Windows/corners.npy', allow_pickle=True)
lbh=np.load('./Windows/lbh.npy', allow_pickle=True)
extras=np.load('./Windows/extras.npy', allow_pickle=True)
heading_angle =np.load('./Windows/heading_angle.npy',allow_pickle=True)
residual_centers=np.load('./Windows/res_centerlabels.npy',  allow_pickle=True)
padded_voxels=np.load('./Windows/voxels.npy', allow_pickle=True)

In [ ]:
residual_lbh=np.array([x[6] for x in extras])
residual_angle=np.array([x[4] for x in extras])
angle_class=np.array([x[3] for x in extras])

In [ ]:
residual_angle[0]

In [ ]:
def conv_onehot_angle_bin(x):
    temp=np.zeros((12))
    temp[x]=1
    return temp
one_hot_angle_class=[conv_onehot_angle_bin(x) for x in angle_class]
residual_angle_labels=[one_hot_angle_class[i]*residual_angle[i] for i in range(residual_angle.shape[0])]

In [ ]:
one_hot_angle_class[0]

In [ ]:
residual_angle_labels[0]

In [ ]:
residual_centers[0]

In [ ]:
np.mean(np.linalg.norm(residual_lbh,axis=1))

In [ ]:
#def create_lbl(i):
#    temp=np.zeros(30)
#    temp[:]
    
targets = np.zeros((len(residual_angle_labels),30))
targets[:,:3] = residual_centers
targets[:,3:6] = residual_lbh
targets[:,6:18] = one_hot_angle_class
targets[:,18:30] = residual_angle_labels

In [ ]:
def boundingBoxRegression(inp):
    conv3d1=tf_util.conv3d(inp,num_output_channels=16,kernel_size=[2,2,2], scope='convr1', bn=True, is_training = True, bn_decay=True)
    #conv3d1 = tf_util.max_pool3d(conv3d1, [2,2,2], scope='maxpool1', stride= [2,2,2])
    conv3d11=tf_util.conv3d(inp,num_output_channels=16,kernel_size=[1,2,1], scope='convr11', bn=True, is_training = True, bn_decay=True)
    conv3d1o=tf.concat([conv3d1,conv3d11 ],axis=4)
    conv3d11 = tf_util.max_pool3d(conv3d1o, [2,2,2], scope='maxpool1', stride= [2,2,2])
    
    conv3d2=tf_util.conv3d(conv3d1,num_output_channels=128,kernel_size=[3,3,3], scope='convr2', bn=True, is_training = True, bn_decay=True)
    #conv3d2 = tf_util.max_pool3d(conv3d2, [2,2,2], scope='maxpool2', stride= [2,2,2])
    conv3d3=tf_util.conv3d(conv3d2,num_output_channels=256,kernel_size=[2,3,2], scope='convr3', bn=True, is_training = True, bn_decay=True)
    
    #deconv3d1=tf_util.conv2d_transpose(conv3d1,128,kernel_size=[1,1])
    #deconv3d2=tf_util.conv2d_transpose(conv3d2,128,)
    
    
    
    dense1 = tf.contrib.layers.flatten(conv3d3)
    dense2 = tf_util.fully_connected(dense1, 64, scope='denser2', bn=True, is_training=True, bn_decay=True)
    #dense3 = tf_util.fully_connected(dense2,128 , scope='denser3', bn=True, is_training=True, bn_decay=True)
    dense4 = tf_util.fully_connected(dense2, 30, activation_fn=None, scope='denser4')
    
    
    return dense4,conv3d1o

In [ ]:
from model_util import g_type_mean_size

In [ ]:
targets.shape

In [ ]:
tf.reset_default_graph()

with tf.device('/device:GPU:0'):

    voxel_inputs =tf.placeholder(tf.float32,[None,30,45,20, 1])#put placeholder

    regress_labels =tf.placeholder(tf.float32,[None,30])

    out,abc = boundingBoxRegression(voxel_inputs)
    abc=abc.get_shape()
    #loss = tf.losses.mean_squared_error(regress_labels, out, scope='loss')
    center_loss1=tf.norm(out-regress_labels,axis=1)#out[0:3]
    center_loss=tf.reduce_mean(center_loss1)
    center_hloss=huber_loss(center_loss1,delta=1.0)
    
    
    residual_sizes_labels=regress_labels[:,3:6]
    
    #residual_size_labels_normalized=tf.div(residual_sizes_labels,tf.constant(g_type_mean_size['Pedestrian'],dtype=tf.float32))
    size_loss1=tf.norm(out[:,3:6]-residual_sizes_labels, axis=1)

    #size_loss1=tf.norm(out[:,3:6]-residual_size_labels_normalized, axis=1)
    #size_hloss=huber_loss(size_loss1,delta=1.0)
    size_loss=tf.reduce_mean(size_loss1)
    
    
    
    
    heading_class_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=regress_labels[:,6:18],logits=out[:,6:18]))
    
    heading_residual_label=regress_labels[:,18:30]
    heading_residual_normalized_label=heading_residual_label/(np.pi/12)
    heading_hloss = huber_loss(tf.reduce_sum(tf.norm(out[:,18:30]-heading_residual_normalized_label,axis=1)*tf.to_float(regress_labels[:,6:18]), axis=1), delta=1.0)
    ####loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.pow(out-regress_labels,2), axis=1)))
    
                                           
    #corner loss
                                         
    # Volume loss
    
    loss=1*size_loss#+0*heading_hloss+0*heading_class_loss+0*center_hloss

    #loss=0*center_hloss+1*size_hloss+0*heading_hloss+0*heading_class_loss
    optimizer = tf.train.AdamOptimizer(1e-3)
    #train_step=optimizer.minimize(loss)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)

In [ ]:
pred_center = {}
test_pred_center = {}
all_losses_train = {}
all_losses_test={}
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=False, log_device_placement=True)) as sess:

    sess.run(tf.global_variables_initializer())
    #variables_names = [v.name for v in tf.trainable_variables()]
    #values = sess.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
    # uncomment this when we want to load the checkpoint model
    #saver.restore(sess, "./checkpoints/regression_"+str(0)+'_'+str(4100)+".ckpt")
    vox = np.load('./Windows/voxels.npy')
    #labels = np.load('./Windows/centerlabels.npy')
    assert vox.shape[0]==targets.shape[0]
    assert vox[0].shape==(30,45,20)
    
    for epoch in range(40):
        print('**************************************************************************************')
        i=0
        batch_size=1
        pred_center[epoch] = []
        all_losses_train[epoch] = []
        while(i<(vox.shape[0]-2000)):
            _, l,s_loss,c_loss,a, b,hc_loss, pred = sess.run([train_op, loss, size_loss, center_loss,size_loss1,residual_sizes_labels, heading_class_loss, out], feed_dict={voxel_inputs:np.reshape(vox[i:i+batch_size], (batch_size, vox[i:i+batch_size].shape[1],\
                                                                                        vox[i:i+batch_size].shape[2], vox[i:i+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(targets[i:i+batch_size],(batch_size,-1))})
#             _, l,s_loss,c_loss,a, b,hc_loss, pred = sess.run([train_op, loss, size_loss, center_loss,size_loss1,residual_sizes_labels, heading_class_loss, out], feed_dict={voxel_inputs:np.reshape(vox[i:i+batch_size], (batch_size, vox[i:i+batch_size].shape[1],\
#                                                                                         vox[i:i+batch_size].shape[2], vox[i:i+batch_size].shape[3], 1)),\
#                                                        regress_labels: np.reshape(targets[i:i+batch_size],(batch_size,-1))})
            i+=batch_size
            #print('epoch=',epoch,'batch=',i,'loss=',l)
            #print('OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
            #print(a)
            #print(b)
            #print(pred[:,3:6])
            print(l)
            #print('OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
            pred_center[epoch].append([pred,i,l])
            all_losses_train[epoch].append([s_loss,c_loss,hc_loss])
            if i%100<5:
                save_path = saver.save(sess, "./checkpoints/regression_"+str(epoch)+'_'+str(i)+".ckpt")
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(all_losses_train[epoch]),axis=0))
                np.save('./Windows/train_pred_'+str(epoch), np.array(pred_center[epoch]), allow_pickle=True)
                np.save('./Windows/train_loss_'+str(epoch), np.array(all_losses_train[epoch]), allow_pickle=True)
                
                #print(np.mean(np.array(pred_center[epoch]), axis=0)[2])
        test=vox.shape[0]-2000
        test_pred_center[epoch]=[]
        all_losses_test[epoch] = []

        while(test<(vox.shape[0])):
            l,s_loss,c_loss,hc_loss, pred = sess.run([loss, size_loss, center_loss, heading_class_loss,out], feed_dict={voxel_inputs:np.reshape(vox[test:test+batch_size], (batch_size, vox[test:test+batch_size].shape[1],\
                                                                                        vox[test:test+batch_size].shape[2], vox[test:test+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(targets[test:test+batch_size],(batch_size,-1))})
            test+=batch_size
            print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
            print('epoch=',epoch,'batch=',test,'loss=',l)
            test_pred_center[epoch].append([pred,test,l])
            all_losses_test[epoch].append([s_loss,c_loss,hc_loss])
            np.save('./Windows/test_pred_'+str(epoch), np.array(test_pred_center[epoch]), allow_pickle=True)
            np.save('./Windows/test_loss_'+str(epoch), np.array(all_losses_test[epoch]), allow_pickle=True)
            if test%50<5:
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(all_losses_test[epoch]),axis=0))

                #print([l,s_loss,c_loss,hc_loss])
                
                #print(np.mean(np.array(test_pred_center[epoch]), axis=0)[2])

In [ ]:
all_losses_train[0][0]

In [ ]:
pred_center = {}
test_pred_center = {}
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:

    sess.run(tf.global_variables_initializer())
    #variables_names = [v.name for v in tf.trainable_variables()]
    #values = sess.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
    # uncomment this when we want to load the checkpoint model
    saver.restore(sess, "./checkpoints/regression_"+str(15)+'_'+str(5000)+".ckpt")
    vox = np.load('./Windows/voxels.npy')
    labels = np.load('./Windows/lbh.npy')
    assert vox.shape[0]==labels.shape[0]
    assert vox[0].shape==(30,45,20)
    
    for epoch in range(40):
        print('**************************************************************************************')
        i=0
        batch_size=5
        pred_center[epoch] = []
        while(i<(vox.shape[0]-2000)):
            _, l, center = sess.run([train_op, loss, out], feed_dict={voxel_inputs:np.reshape(vox[i:i+batch_size], (batch_size, vox[i:i+batch_size].shape[1],\
                                                                                        vox[i:i+batch_size].shape[2], vox[i:i+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(labels[i:i+batch_size],(batch_size,-1))})
            i+=batch_size
            print('epoch=',epoch,'batch=',i,'loss=',l)
            pred_center[epoch].append([center,i,l])
            if i%1000==0:
                save_path = saver.save(sess, "./checkpoints/lbhregression_"+str(epoch)+'_'+str(i)+".ckpt")
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(pred_center[epoch]), axis=0)[2])
        test=vox.shape[0]-2000
        test_pred_center[epoch]=[]
        while(test<(vox.shape[0])):
            l, center = sess.run([loss, out], feed_dict={voxel_inputs:np.reshape(vox[test:test+batch_size], (batch_size, vox[test:test+batch_size].shape[1],\
                                                                                        vox[test:test+batch_size].shape[2], vox[test:test+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(labels[test:test+batch_size],(batch_size,-1))})
            test+=batch_size
            print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
            print('epoch=',epoch,'batch=',test,'loss=',l)
            test_pred_center[epoch].append([center,test,l])
            if test%500<5:
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(test_pred_center[epoch]), axis=0)[2])

In [ ]:
#huber loss
#gradient clipping


In [ ]:
labels.shape

In [ ]:
plt.hist(labels[:,0])

In [ ]:
np.std(labels[:,0])

In [ ]:
plt.hist(labels[:,1])

In [ ]:
np.std(labels[:,1])

In [ ]:
plt.hist(labels[:,2])

In [ ]:
np.std(labels[:,2])